In [1]:
from utils.ReviewRBInit import ReviewRBInit
from utils.ReviewRBLoop import ReviewRBLoop
from utils.ReviewMLInit import ReviewMLInit
from utils.ReviewMLLoop import ReviewMLLoop
from gui.Workflow import Workflow
from sqlalchemy_dao import Dao
from db.ORMs import Document
from utils.IntroStep import IntroStep
from gui.PreviousNextWidgets import PreviousNextHTML
import sqlalchemy_dao
import os
from  conf.ConfigReader import ConfigReader
from models.logistic.LogisticBOWClassifier import LogisticBOWClassifier

import logging
logging.getLogger().setLevel(logging.DEBUG)

ConfigReader('../conf/smartanno_conf.json')
wf=Workflow()
rb=ReviewRBInit(name="rb_review_init")
wf.append(rb)
rv=ReviewRBLoop(name='rb_review')
wf.append(rv)
wf.append(PreviousNextHTML('<h2>Congratuations!</h2><h4>You have finished the initial review on the rule-base preannotations. </h4>',
                       name='intro'))
wf.append(ReviewMLInit(name='ml_review_init'))
wf.append(ReviewMLLoop(name='ml_review',ml_classifier_cls=LogisticBOWClassifier))

wf.filters={'TypeA':['heart'],'TypeB':['exam']}
wf.types=['TypeA','TypeB']
wf.task_id=1
wf.umls_extended={}
wf.we_extended={}
wf.dao=Dao('sqlite+pysqlite:///../data/demo.sqlite', sqlalchemy_dao.POOL_DISABLED)

ToggleButtonsMultiSelectionInBox(children=(VBox(layout=Layout(display='flex')),), _dom_classes=('btn-group',))

In [2]:
wf.start()

VBox(children=(IntProgress(value=15, description='Progress: 15/19', layout=Layout(height='14px', width='90%'), max=19, min=1, style=ProgressStyle(description_width='initial')), HTML(value='<table style="width:100%"><tr><th>DOC_ID</th><th>DATE</th><th>REF_DATE</th></tr><tr><td>34</td><td>2072-01-01</td><td>None</td></tr></table><div id="d1" style="overflow-y: scroll; height:200px;border:1px solid;border-color:#e5e8e8; ">Record date: 2072-01-01<br/><br/>EDVISIT^84091519^THOMAS-YOSEF, JULIA^01/01/72^HOFFMAN, VICTOR<br/><br/>Seen by me along with Dr. Ickes on 01/01/2072.  History as<br/><br/>documented by Dr. Ickes was reviewed by me with the patient and<br/><br/>found to be accurate and complete.  The physical <span class="highlighter" style="background-color:  None ">exam</span> was<br/><br/>confirmed by my examination.  Evaluation and management was also<br/><br/>concurred with.<br/><br/>HISTORY OF PRESENT ILLNESS:  In brief, this is a 51-year-old<br/><br/>woman with history of coronary artery disease status post MI<br/><br/>stenting in 2068, presents with 3-day history of substernal chest<br/><br/>pain radiating to her back, and also complaining of mild pain of<br/><br/>inner medial aspect of her right arm, feels like her anginal<br/><br/>symptoms that she had in 2068 with some very mild shortness of<br/><br/>breath and even though she is quite sedentary and she noted that<br/><br/>there is an increase in pain with exertion, position, as well as<br/><br/>eating.  She took many Tums with no relief and says takes Zantac<br/><br/>chronically, but ran out of it.  She says that the pain is<br/><br/>burning epigastric at times, and the thing that is concerning was<br/><br/>her back pain that was reminiscent of the pain that she had when<br/><br/>she had her MI.<br/><br/>PAST MEDICAL HISTORY:  She has a history of hypertension,<br/><br/>diabetes, MI, and breast cancer status post lumpectomy and<br/><br/>radiation.  Last stress test was may be in October, which was<br/><br/>normal.<br/><br/>FAMILY HISTORY:  Positive for breast cancer and diabetes<br/><br/>mellitus.<br/><br/>SOCIAL HISTORY:  She does not drink.  She smoked in the past,<br/><br/>quit 15 years ago.<br/><br/>REVIEW OF SYSTEMS:  As documented in the chart.  Please see notes<br/><br/>for details.<br/><br/>ALLERGIES:  As documented in the chart.  Please see notes for<br/><br/>details.<br/><br/>MEDICATIONS:  As documented in the chart.  Please see notes for<br/><br/>details.<br/><br/>PHYSICAL EXAMINATION:  Awake, alert, well-developed,<br/><br/>well-nourished woman, and in no apparent distress.  Afebrile,<br/><br/>slightly 96.7, pulse 72, respirations 16, blood pressure 124/72,<br/><br/>and O2 saturation 98%.  Anicteric sclerae.  Mucous membranes are<br/><br/>moist.  Neck is supple.  Lungs are clear of rales, wheezes, or<br/><br/>rhonchi.  Cardiovascular:  Regular rate and rhythm.  Abdomen is<br/><br/>obese.  Bowel sounds normoactive, nontender, and no guarding,<br/><br/>rebound, or rigidity.  The skin is intact, warm, and dry.  She<br/><br/>has point tenderness over the upper thoracic cage and trace pedal<br/><br/>edema.<br/><br/>LABORATORY DATA:  The patient has a normal CBC and normal<br/><br/>electrolytes.  D-dimer was less than 200.  Troponin and CK were<br/><br/>normal.<br/><br/>IMAGING STUDIES:  EKG sinus rhythm with unchanged from previous. <br/><br/>Chest CT was negative PE.<br/><br/>EMERGENCY DEPARTMENT COURSE:  As such, she was admitted to the<br/><br/>Observation Unit for a second set of enzymes which if normal will<br/><br/>be discharged home, after walking to see if she has any<br/><br/>exertional component, as such, she was subsequently discharged to<br/><br/>home in stable and satisfactory condition.  She was admitted to<br/><br/>the Observation Unit 01/01/2072 with diagnosis of chest pain.<br/><br/>______________________________<br/><br/>HOFFMAN, VICTOR M.D.<br/><br/>D: 01/01/72<br/><br/>T: 01/02/72<br/><br/>Dictated By: HOFFMAN, VICTOR<br/><br/>eScription do

In [3]:
[step.doc.DOC_ID for step in wf.steps[4].loop_workflow.steps if hasattr(step, "doc")]

[]

In [4]:
[(doc.DOC_ID,doc.DOC_NAME) for doc in wf.steps[4].docs]

[]

In [5]:
[step.reviewed for step in wf.steps[4].loop_workflow.steps]

[]

In [6]:
from models.logistic.LogisticBOWClassifier import LogisticBOWClassifier
LogisticBOWClassifier.status

0

In [7]:
reviewed = {doc_id: anno.REVIEWED_TYPE for doc_id, anno in wf.steps[4].data['annos'].items() if anno.REVIEWED_TYPE is not None}

KeyError: 'annos'

In [ ]:
[doc.DOC_ID for doc in wf.steps[4].docs]

In [ ]:
[doc.TEXT for doc in wf.steps[4].docs if doc.DOC_ID in wf.steps[4].reviewed]

In [ ]:
x = [doc.TEXT for doc in wf.steps[4].docs[:len(wf.steps[4].reviewed)]]
y = list(wf.steps[4].reviewed.values())

In [ ]:
wf.steps[4].reviewed

In [ ]:
LogisticBOWClassifier.instance.train(x,y)

In [ ]:
ml.model.train